# ETL Pipeline for Sparkify, a Music Streaming App, Using an Apache Cassandra Keyspace

The fictional company, Sparkify, wants to analyze the data they've been collecting on songs and user activity on their new music streaming app. The analysis team is particularly interested in understanding what songs users are listening to. 

We'll create an Apache Cassandra database which can create queries on song play data to enable the analytics team answer their questions.

## First, We Create an ETL Pipeline for Pre-Processing the Files
Sparkify stores this event data in a directory of CSV files. We'll preprocess these files with some python code before modeling the data by inserting them into Apache Cassandra tables.

#### Import Python packages 

In [33]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event CSV data files

In [34]:
# checking the current working directory
print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# Join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file CSV that will be used for Apache Casssandra tables

In [35]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and appending it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [36]:
# Check the number of rows in the new csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## Now we can get down to writing some Apache Cassandra code 

The CSV file titled we've just created <font color=red>event_datafile_new.csv</font> contains the denormalized data from which we'll create tables to run specific queries.


#### Creating a Cluster

In [37]:
# This creates a connection to a Cassandra instance in your local machine (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, a session is need.
session = cluster.connect()

#### Create Keyspace

In [38]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb
    WITH REPLICATION = {'class' : 'SimpleStrategy', 'replication_factor' : 1}
    """)
except Exception as e:
    print(e)    

#### Set Keyspace

In [39]:
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

#### Set RowFactory to Convert Apache Cassandra Row Results to Pandas DataFrame

In [40]:
def pandas_factory(colnames, rows):
    "Returns a Pandas DataFrame with named columns"
    return pd.DataFrame(rows, columns=colnames)

try:
    session.row_factory = pandas_factory
except Exception as e:
    print(e)

## We'll create queries to ask the following three questions of the data:

#### 1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4


#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'





### Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

Create a table for Query 1 using both sessionId, itemInSession to create a composite partition key in the PRIMARY KEY to uniquely identify each row in the table and also avoid using ALLOW FILTERING when running the SELECT query.

In [41]:
query = """
        CREATE TABLE IF NOT EXISTS song_session (
            sessionId int, 
            itemInSession int, 
            artist text, 
            song text, 
            length float,
            PRIMARY KEY (sessionId, itemInSession))
        """
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [42]:
# Apache Cassandra code to set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
## Assigning the INSERT statements into the `query` variable
        query = """
                INSERT INTO song_session (
                    sessionId, 
                    itemInSession, 
                    artist, 
                    song, 
                    length)
                VALUES (%s, %s, %s, %s, %s)
                """
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### SELECT statement to verify the data was entered into the table

In [43]:
query = """
        SELECT artist, song, length 
        FROM song_session 
        WHERE sessionId = 338 AND itemInSession = 4
        """
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

rows._current_rows

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### Query 2: We need only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

Create another table for Query 2 using both userId and sessionId to create a composite partition key in the PRIMARY KEY to uniquely identify each row in the table and also avoid using ALLOW FILTERING when running the SELECT query.

The PRIMARY KEY also includes a clustering column (itemInSession) which will sort songs in ascending order

In [44]:
query = """
        CREATE TABLE IF NOT EXISTS song_playlist_session (
            userId int, 
            sessionId int, 
            itemInSession int, 
            artist text, 
            song text,
            firstName text, 
            lastName text, 
            PRIMARY KEY ((userId, sessionId), itemInSession))
        """
try:
    session.execute(query)
except Exception as e:
    print(e)
    
# Apache Cassandra code to set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
## Assigning the INSERT statements into the `query` variable
        query = """
                INSERT INTO song_playlist_session (
                    userId, 
                    sessionId, 
                    itemInSession, 
                    artist, 
                    song, 
                    firstName, 
                    lastName)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
                """
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

## SELECT statement to verify the data was entered into the table
query = """
        SELECT artist, song, firstName, lastName 
        FROM song_playlist_session 
        WHERE userid = 10 AND sessionid = 182
        """

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
rows._current_rows
                    

,artist,song,firstname,lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### Query 3: Find every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Create another table for Query 3 using song along with userId (defined as a clustering column) to create a compound PRIMARY KEY to uniquely identify each row in the table.


In [45]:
query = """
        CREATE TABLE IF NOT EXISTS user_song_playlist (
            song text,
            userId int,
            firstName text, 
            lastName text,  
            PRIMARY KEY ((song), userId))
        """
try:
    session.execute(query)
except Exception as e:
    print(e)
    
# Apache Cassandra code to set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
## Assigning the INSERT statements into the `query` variable
        query = """
                INSERT INTO user_song_playlist (
                    song,
                    userId,
                    firstName, 
                    lastName)
                VALUES (%s, %s, %s, %s)
                """
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

## SELECT statement to verify the data was entered into the table
query = """
        SELECT firstName, lastName 
        FROM user_song_playlist 
        WHERE song = 'All Hands Against His Own'
        """
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
rows._current_rows                   

,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [46]:
## Drop all tables before closing out the sessions
query = "DROP TABLE IF EXISTS song_session"
query1 = "DROP TABLE IF EXISTS song_playlist_session"
query2 = "DROP TABLE IF EXISTS user_song_playlist"

try:
    session.execute(query)
    session.execute(query1)
    session.execute(query2)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [47]:
session.shutdown()
cluster.shutdown()